First, let's create a synthetic dataset:

In [1]:
from collections import Counter
from sklearn.datasets import make_classification
# create dataframe
X, y = make_classification(n_samples=1000,
                           n_features=20,
                           n_informative=15,
                           n_redundant=5,
                           random_state=1)
# print the data classes info
print(f'''Main dataframe:
Number of samples: {X.shape[0]}
Number of features: {X.shape[1]}
Samples by class:''')
counter = Counter(y)
for k, v in counter.items():
    per = v / len(y) * 100
    print('Class=%d, Count=%d, Percentage=%.1f%%' % (k, v, per))
    
# Main dataframe:
# Number of samples: 1000
# Number of features: 20
# Samples by class:
# Class=0, Count=501, Percentage=50.1%
# Class=1, Count=499, Percentage=49.9%

Then, manage with a simple train / test split:

In [2]:
from sklearn.model_selection import train_test_split
# split data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, stratify=y, random_state=1)
# show the data classes info
print(f'''\nTrain dataframe:
Number of samples: {X_train.shape[0]}
Number of features: {X_train.shape[1]}
Samples by class:''')
counter = Counter(y_train)
for k, v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%d, Count=%d, Percentage=%.1f%%' % (k, v, per))

# Train dataframe:
# Number of samples: 500
# Number of features: 20
# Samples by class:
# Class=0, Count=251, Percentage=50.2%
# Class=1, Count=249, Percentage=49.8%

Write a function to fit and evaluate the models:

In [3]:
import time
from sklearn.metrics import accuracy_score, confusion_matrix
# define model evaluator
def predictor(model):
    tic = time.perf_counter()
    # fit classifier
    model.fit(X_train, y_train)
    # get prediction
    y_pred = model.predict(X_test)
    # check results
    accuracy = accuracy_score(y_test, y_pred)
    matrix = confusion_matrix(y_test, y_pred)
    # show results
    print(f'''\n{type(model).__name__} results:
Accuracy:{accuracy * 100: 0.1f}%
{matrix}''')
    toc = time.perf_counter()
    print(f"Processed in {toc - tic: 0.4f} seconds")

Let's look at the models:

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# define model a and model b
model_a = RandomForestClassifier(random_state=1)
model_b = KNeighborsClassifier()
# fit and evaluate models
predictor(model_a)
predictor(model_b)

# RandomForestClassifier results:
# Accuracy: 91.2%
# [[234  16]
#  [ 28 222]]
# Processed in  0.1965 seconds
#
# KNeighborsClassifier results:
# Accuracy: 90.4%
# [[223  27]
#  [ 21 229]]
# Processed in  0.0781 seconds

Let's predict both classes using the "a" model:

In [5]:
import copy
import pandas as pd
# get prediction of model a
y_pred_a_01 = model_a.predict(X_test)
# define dataframe with prediction of model a
df_pred_a_01 = copy.copy(X_test)
df_pred_a_01 = pd.DataFrame(df_pred_a_01)
df_pred_a_01['y'] = y_pred_a_01.tolist()
# show the data classes info
print(f'''\nDataframe with prediction of model a:
Number of samples: {df_pred_a_01.shape[0]}
Number of features: {df_pred_a_01.shape[1] - 1}
Samples by class:''')
counter = Counter(y_pred_a_01)
for k, v in counter.items():
    per = v / len(y_pred_a_01) * 100
    print('Class=%d, Count=%d, Percentage=%.1f%%' % (k, v, per))

# Dataframe with prediction of model a:
# Number of samples: 500
# Number of features: 20
# Samples by class:
# Class=1, Count=238, Percentage=47.6%
# Class=0, Count=262, Percentage=52.4%

Split dataframe with prediction of model a to predictions of first and second classes

In [6]:
# define with what class model a will operate
class_order = 'first'
# split dataframe with prediction of model a to predictions of first and second classes
if class_order == 'first':
    df_pred_a_f = df_pred_a_01[df_pred_a_01['y'] == 0]
    df_test_b_s = df_pred_a_01[df_pred_a_01['y'] == 1]
if class_order == 'second':
    df_pred_a_f = df_pred_a_01[df_pred_a_01['y'] == 1]
    df_test_b_s = df_pred_a_01[df_pred_a_01['y'] == 0]
# define test dataframe of model b with predicted samples of chosen class
X_test_b_s = df_test_b_s.drop(['y'], axis=1)
# get prediction of model b
y_pred_b_s = model_b.predict(X_test_b_s)
# define dataset of prediction of model b
df_pred_b_s = copy.copy(X_test_b_s)
df_pred_b_s['y'] = y_pred_b_s.tolist()
print(f'''\nDataframe with prediction of model b:
Number of samples: {df_pred_b_s.shape[0]}
Number of features: {df_pred_b_s.shape[1] - 1}
Samples by class:''')
counter = Counter(y_pred_b_s)
for k, v in counter.items():
    per = v / len(y_pred_b_s) * 100
    print('Class=%d, Count=%d, Percentage=%.1f%%' % (k, v, per))

# Dataframe with prediction of model b:
# Number of samples: 238
# Number of features: 20
# Samples by class:
# Class=0, Count=19, Percentage=8.0%
# Class=1, Count=219, Percentage=92.0%

To calculate the overall accuracy of the two models and compare with how they coped alone combine the results:

In [7]:
# add dataframes with predictions of model a and b together
df_pred_ab_01 = pd.concat([df_pred_a_f, df_pred_b_s]).sort_index()
y_pred_ab_01 = df_pred_ab_01['y']
# get accuracy and confusion matrix of predictions of model a and b
accuracy_ab_01 = accuracy_score(y_test, y_pred_ab_01)
matrix_ab_01 = confusion_matrix(y_test, y_pred_ab_01)
# show results
print(f'''\nModel a and b results {class_order} class first:
Accuracy:{accuracy_ab_01 * 100: 0.1f}%
{matrix_ab_01}''')

# Model a and b results first class first:
# Accuracy: 90.2%
# [[241   9]
#  [ 40 210]]

Let's add the method above into one function and let the model "b" now work not with class 1, but with class 0:

In [8]:
# Evaluate models on different classes
def class_in_order_predictor(class_order='first'):
    print('\nCurrent chosen class for model b:', class_order)
    # get prediction of model a
    y_pred_a_01 = model_a.predict(X_test)
    # define dataframe with prediction of model a
    df_pred_a_01 = copy.copy(X_test)
    df_pred_a_01 = pd.DataFrame(df_pred_a_01)
    df_pred_a_01['y'] = y_pred_a_01.tolist()
    # show the data classes info
    print(f'''\nDataframe with prediction of model a:
Number of samples: {df_pred_a_01.shape[0]}
Number of features: {df_pred_a_01.shape[1] - 1}
Samples by class:''')
    counter = Counter(y_pred_a_01)
    for k, v in counter.items():
        per = v / len(y_pred_a_01) * 100
        print('Class=%d, Count=%d, Percentage=%.1f%%' % (k, v, per))
    # split dataframe with prediction of model a to predictions of first and second classes
    if class_order == 'first':
        df_pred_a_f = df_pred_a_01[df_pred_a_01['y'] == 0]
        df_test_b_s = df_pred_a_01[df_pred_a_01['y'] == 1]
    if class_order == 'second':
        df_pred_a_f = df_pred_a_01[df_pred_a_01['y'] == 1]
        df_test_b_s = df_pred_a_01[df_pred_a_01['y'] == 0]
    # define test dataframe of model b with predicted samples of second class
    X_test_b_s = df_test_b_s.drop(['y'], axis=1)
    # get prediction of model b
    y_pred_b_s = model_b.predict(X_test_b_s)
    # define dataset of prediction of model b
    df_pred_b_s = copy.copy(X_test_b_s)
    df_pred_b_s['y'] = y_pred_b_s.tolist()
    print(f'''\nDataframe with prediction of model b:
Number of samples: {df_pred_b_s.shape[0]}
Number of features: {df_pred_b_s.shape[1] - 1}
Samples by class:''')
    counter = Counter(y_pred_b_s)
    for k, v in counter.items():
        per = v / len(y_pred_b_s) * 100
        print('Class=%d, Count=%d, Percentage=%.1f%%' % (k, v, per))
    # add dataframes with predictions of model a and b together
    df_pred_ab_01 = pd.concat([df_pred_a_f, df_pred_b_s]).sort_index()
    y_pred_ab_01 = df_pred_ab_01['y']
    # get accuracy and confusion matrix of predictions of model a and b
    accuracy_ab_01 = accuracy_score(y_test, y_pred_ab_01)
    matrix_ab_01 = confusion_matrix(y_test, y_pred_ab_01)
    # show results
    print(f'''\nModel a and b results {class_order} class first:
Accuracy:{accuracy_ab_01 * 100: 0.1f}%
{matrix_ab_01}''')
# get result with second class for model b
class_in_order_predictor(class_order='second')

# Current chosen class for model b: second
#
# Dataframe with prediction of model a:
# Number of samples: 500
# Number of features: 20
# Samples by class:
# Class=1, Count=238, Percentage=47.6%
# Class=0, Count=262, Percentage=52.4%
#
# Dataframe with prediction of model b:
# Number of samples: 262
# Number of features: 20
# Samples by class:
# Class=0, Count=225, Percentage=85.9%
# Class=1, Count=37, Percentage=14.1%
#
# Model a and b results second class first:
# Accuracy: 91.4%
# [[216  34]
#  [  9 241]]

Is it possible to achieve an even better result if the models are swapped? Let's find out!

In [9]:
print("\nSwap models a and b:")
model_a = KNeighborsClassifier().fit(X_train, y_train)
model_b = RandomForestClassifier(random_state=1).fit(X_train, y_train)
# call predictor with independent models for each class
class_in_order_predictor(class_order='second')

# Swap models a and b:
#
# Current chosen class for model b: second
#
# Dataframe with prediction of model a:
# Number of samples: 500
# Number of features: 20
# Samples by class:
# Class=0, Count=244, Percentage=48.8%
# Class=1, Count=256, Percentage=51.2%
#
# Dataframe with prediction of model b:
# Number of samples: 244
# Number of features: 20
# Samples by class:
# Class=1, Count=19, Percentage=7.8%
# Class=0, Count=225, Percentage=92.2%
#
# Model a and b results second class first:
# Accuracy: 91.4%
# [[216  34]
#  [  9 241]]

And now let's compare the result with the work of Voting ensembles:

In [10]:
from sklearn.ensemble import VotingClassifier
# call hard voting ensemble
predictor(VotingClassifier(estimators=[('a', model_a), ('b', model_b)], voting='hard'))

# VotingClassifier results:
# Accuracy: 90.2%
# [[241   9]
#  [ 40 210]]
# Processed in  0.1990 seconds

A Hard voting showed exactly the result achieved when giving model "b" a class 1 job. Soft voting showed the best result among all models and approaches:

In [11]:
# call soft voting ensemble
predictor(VotingClassifier(estimators=[('a', model_a), ('b', model_b)], voting='soft'))

# VotingClassifier results:
# Accuracy: 91.6%
# [[230  20]
#  [ 22 228]]
# Processed in  0.2079 seconds